In [17]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
import os

import sys
sys.path.insert(0, '/Users/guilhermeosorio/Desktop/ARGO/argo-backend')

from app.utils.langchain import DocPath, generate_text_from_path, generate_chunks
from app.config import app_config
from app.utils.embeddings import EmbeddingRequest, UpsertRequest, generate_embedding, upsert_embedding
from app.utils.pinecone import pinecone_client
from app.models.vectordb import Vector, VectorMetaData

from dotenv import load_dotenv
load_dotenv()

CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX_HOST = os.getenv('PINECONE_INDEX_HOST')

In [36]:
directory = '../../_AI Input Proposals'

def list_rfp_files(directory):
    rfp_files = []
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename == "RFP.pdf":
                rfp_files.append(os.path.join(dirpath, filename))
    return rfp_files

directory += '/Known'
rfp_paths = list_rfp_files(directory)
rfp_paths


['../../_AI Input Proposals/Known/VFPA Gateway/RFP.pdf',
 '../../_AI Input Proposals/Known/Cullen Commission/RFP.pdf',
 '../../_AI Input Proposals/Known/Canada Energy Regulator/RFP.pdf',
 '../../_AI Input Proposals/Known/City of Castlegar/RFP.pdf',
 '../../_AI Input Proposals/Known/Broadway Subway Project/RFP.pdf',
 '../../_AI Input Proposals/Known/BC Housing Strat Com/RFP.pdf',
 '../../_AI Input Proposals/Known/Metro Vancouver/RFP.pdf',
 '../../_AI Input Proposals/Known/CoV - Creative Design/RFP.pdf',
 '../../_AI Input Proposals/Known/DWV/RFP.pdf',
 '../../_AI Input Proposals/Known/Cowichan Tribes/RFP.pdf',
 '../../_AI Input Proposals/Known/BC Transit/RFP.pdf',
 '../../_AI Input Proposals/Known/RDN/RFP.pdf']

In [19]:
test_rfp_path = rfp_paths[0] # VFPA Gateway
# pdfs to chunks
input = DocPath(doc_path=test_rfp_path)
text = generate_text_from_path(request=input)
chunks = generate_chunks(request=text).texts
# chunks to pinecone
rfp_name = test_rfp_path.split('/')[-2]

In [20]:
test_rfp = "VFPA Gateway"

In [21]:
llm_small = ChatAnthropic(temperature=0, anthropic_api_key=CLAUDE_API_KEY, model="claude-3-haiku-20240307") # smallest
llm_medium = ChatAnthropic(temperature=0, anthropic_api_key=CLAUDE_API_KEY, model="claude-3-sonnet-20240229") # medium
llm_large = ChatAnthropic(temperature=0, anthropic_api_key=CLAUDE_API_KEY, model="claude-3-opus-20240229") # larger

In [33]:
summarized_chunks = []

for chunk in chunks:
    system = """
    You are an expert consultant analyzing an RFP that was just submitted for your review.
    You will be given a chunk taken out of this RFP document. Your goal is to summarize the given chunk, keeping the most relevant parts of the content.
    When summarizing the RFP chunk, start directly with the summary content, without mentioning 'the given chunk summarizes' or any similar phrasing.
    If any instruction(s) about the response to be submitted to the RFP are present in the chunk, it is crucial that you keep them in your summarization.
    """
    human = """
    Chunk:
    {chunk}

    Summarized chunk:
    """

    RFP_summarizer = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

    chain = RFP_summarizer | llm_small

    summarized_chunk = chain.invoke(
        {
            "chunk": chunk
        }
    )
    summarized_chunks.append(summarized_chunk.content)

In [34]:
for i, r in enumerate(chunks):
    print(f"CHUNK {i}")
    print(f"{r}\n")

CHUNK 0
VANCOUVER FRASER PORT AUTHORITY  

REQUEST FOR PROPOSAL: #P210205-01 

Graphic design support for Gateway projects 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1.0 

NOTICE TO PROPONENTS 

THIS IS AN INVITATION FOR PROPOSALS AND NOT A TENDER CALL. 

(A.) 

(B.) 

Proposals are to be submitted electronically via email (one pdf file attachment preferred) to the 
attention of the contact addresses identified in Section 1.0 (C.) by the closing time of 3:00 p.m. 
local Vancouver time on February 5, 2021.  The subject line should include the Request for 
Proposal (“RFP”) number and title “P210205-01”.  It is the responsibility of the proponent to 
ensure that the emailed proposal is successfully sent and received by the closing time.  Late 
proposals may not be considered. 

The deadline for Questions relating to this RFP is by 5:00 p.m. on January 25, 2021 (“the 
Deadline”). The Deadline will apply to any addendum issued prior to the Deadline, unless a new 
Deadlin

In [23]:
for i, r in enumerate(summarized_chunks):
    print(f"CHUNK {i}")
    print(f"{r}\n")

CHUNK 0
This is a Request for Proposal (RFP) from the Vancouver Fraser Port Authority seeking proposals from graphic designers or agencies to provide support in developing and executing public materials for a suite of infrastructure projects the port authority is leading in 2021. The key points are:

- Proposals must be submitted electronically by 3:00 pm local Vancouver time on February 5, 2021. 
- Deadline for questions is 5:00 pm on January 25, 2021. All enquiries should be emailed to the specified contacts.
- The port authority reserves the right to refuse questions, cancel the RFP process, and not accept any proposal.
- The port authority is undertaking projects to enhance cargo movement, improve mobility and safety, protect the environment, and alleviate impacts of growing trade on local communities.
- These projects are part of the Greater Vancouver Gateway 2030 Strategy developed by the Gateway Transportation Collaboration Forum to identify infrastructure investments for an eff

In [24]:
# Concatenate the strings into a single string
summarized_chunks_str = "\n".join(summarized_chunks)
print(summarized_chunks_str)

This is a Request for Proposal (RFP) from the Vancouver Fraser Port Authority seeking proposals from graphic designers or agencies to provide support in developing and executing public materials for a suite of infrastructure projects the port authority is leading in 2021. The key points are:

- Proposals must be submitted electronically by 3:00 pm local Vancouver time on February 5, 2021. 
- Deadline for questions is 5:00 pm on January 25, 2021. All enquiries should be emailed to the specified contacts.
- The port authority reserves the right to refuse questions, cancel the RFP process, and not accept any proposal.
- The port authority is undertaking projects to enhance cargo movement, improve mobility and safety, protect the environment, and alleviate impacts of growing trade on local communities.
- These projects are part of the Greater Vancouver Gateway 2030 Strategy developed by the Gateway Transportation Collaboration Forum to identify infrastructure investments for an efficient t

In [25]:
system = """
You are an expert consultant analyzing an RFP that was just submitted for your review.
Based on an RFP summary, create a bulleted list to serve as a structure for the response to the RFP.
Your main goal is to provide a bulleted point list to serve as a reference in the response, as the team is looking to automate the process of generating an initial draft to respond to RFPs.
You are part of a company called Kirk & Co., which is one of Canada's leading firms specializing in strategic communications, consultation and engagement.
In the response structure, separate the sections with the substring '\n\n'
"""
human = """
RFP Summary:
{rfp_summary}

Response structure:
"""

response_structurer = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = response_structurer | llm_medium

response_structure = chain.invoke(
    {
        "rfp_summary": summarized_chunks_str
    }
).content


In [26]:
print(response_structure)

Based on the RFP summary, here is a bulleted list structure to serve as a reference for the response:

1.0 Executive Summary
• Brief overview of Kirk & Co. and our qualifications
• Summary of our understanding of the RFP requirements
• Highlights of our proposed approach and key differentiators

2.0 Company Profile
• Background and history of Kirk & Co.
• Core services and areas of expertise
• Relevant experience with similar projects/clients
• Corporate values and commitment to sustainability

3.0 Project Understanding and Approach
• Detailed understanding of the project scope and objectives
• Proposed methodology and work plan
• Project team structure and roles
• Quality assurance and control measures
• Risk management and mitigation strategies

4.0 Project Team
• Profiles of key personnel and their relevant experience
• Roles and responsibilities of each team member
• Capacity and availability to meet project timelines
• Approach to collaboration and communication

5.0 Previous Work

In [27]:
response_structure

"Based on the RFP summary, here is a bulleted list structure to serve as a reference for the response:\n\n1.0 Executive Summary\n• Brief overview of Kirk & Co. and our qualifications\n• Summary of our understanding of the RFP requirements\n• Highlights of our proposed approach and key differentiators\n\n2.0 Company Profile\n• Background and history of Kirk & Co.\n• Core services and areas of expertise\n• Relevant experience with similar projects/clients\n• Corporate values and commitment to sustainability\n\n3.0 Project Understanding and Approach\n• Detailed understanding of the project scope and objectives\n• Proposed methodology and work plan\n• Project team structure and roles\n• Quality assurance and control measures\n• Risk management and mitigation strategies\n\n4.0 Project Team\n• Profiles of key personnel and their relevant experience\n• Roles and responsibilities of each team member\n• Capacity and availability to meet project timelines\n• Approach to collaboration and communi

In [28]:
response_sections = response_structure.split("\n\n")
response_sections

['Based on the RFP summary, here is a bulleted list structure to serve as a reference for the response:',
 '1.0 Executive Summary\n• Brief overview of Kirk & Co. and our qualifications\n• Summary of our understanding of the RFP requirements\n• Highlights of our proposed approach and key differentiators',
 '2.0 Company Profile\n• Background and history of Kirk & Co.\n• Core services and areas of expertise\n• Relevant experience with similar projects/clients\n• Corporate values and commitment to sustainability',
 '3.0 Project Understanding and Approach\n• Detailed understanding of the project scope and objectives\n• Proposed methodology and work plan\n• Project team structure and roles\n• Quality assurance and control measures\n• Risk management and mitigation strategies',
 '4.0 Project Team\n• Profiles of key personnel and their relevant experience\n• Roles and responsibilities of each team member\n• Capacity and availability to meet project timelines\n• Approach to collaboration and co

In [29]:
system = """
You are an expert consultant working on drafting a response to an RFP.
Based on an RFP summary and a given outline for a section of the response, develop the section's content.
Your main goal is to provide the final draft for this section. After all sections have been generated, they will be concatenated in the final draft of the response.
You are part of a company called Kirk & Co., which is one of Canada's leading firms specializing in strategic communications, consultation and engagement.
"""
human = """
RFP Summary:
{rfp_summary}

Section outline:
{section_structure}

Section:
"""

response_composer = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = response_composer | llm_medium

final_response_sections = []

for section in response_sections:


    section_content = chain.invoke(
        {
            "rfp_summary": summarized_chunks_str,
            "section_structure": section
        }
    ).content
    final_response_sections.append(section_content)

In [30]:
for i, r in enumerate(final_response_sections):
    print(f"SECTION {i}")
    print(f"{r}\n")

SECTION 0
Based on the RFP summary and the provided outline, here is a draft for the section on the Proposed Services, Methodology, and Approach:

Section: Proposed Services, Methodology, and Approach

• Overview of Services
   - Graphic design and communications support for infrastructure projects
   - Developing creative concepts and designs aligned with VFPA brand guidelines  
   - Producing materials like infographics, maps, ads, presentations, videos, reports
   - Capturing photographs, creating renderings, and editing/proofreading materials
   - Coordinating with third-party vendors for printing, publishing, etc.

• Methodology and Approach
   - Kick-off meeting to review project requirements, timelines, and VFPA guidelines
   - Develop creative briefs and concepts for each project 
   - Iterate on designs based on feedback from VFPA team
   - Ensure consistent branding and messaging across all materials
   - Leverage expertise in infrastructure/transportation for compelling visu

In [31]:
final_response_sections_str = "\n".join(final_response_sections)
print(final_response_sections_str)

Based on the RFP summary and the provided outline, here is a draft for the section on the Proposed Services, Methodology, and Approach:

Section: Proposed Services, Methodology, and Approach

• Overview of Services
   - Graphic design and communications support for infrastructure projects
   - Developing creative concepts and designs aligned with VFPA brand guidelines  
   - Producing materials like infographics, maps, ads, presentations, videos, reports
   - Capturing photographs, creating renderings, and editing/proofreading materials
   - Coordinating with third-party vendors for printing, publishing, etc.

• Methodology and Approach
   - Kick-off meeting to review project requirements, timelines, and VFPA guidelines
   - Develop creative briefs and concepts for each project 
   - Iterate on designs based on feedback from VFPA team
   - Ensure consistent branding and messaging across all materials
   - Leverage expertise in infrastructure/transportation for compelling visuals
   - U

In [32]:
len(final_response_sections_str)

24893